In [3]:
# !pip install tensorflow
# !pip  install moabb
# !pip install imblearn

In [31]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os, moabb,sys, time,pickle
from moabb.datasets import BNCI2014001
import pandas as pd
from cnn_utils import *
import warnings
warnings.filterwarnings('ignore')
import tensorflow.compat.v1 as tf
import numpy as np
import scipy.io as siobn 
from moabb.pipelines.utils import FilterBank
tf.disable_v2_behavior()
from sklearn.pipeline import make_pipeline,Pipeline
from pyriemann.estimation import Covariances

##########

# saving the results
def save_csvfile(results,sfile_name):
    if(os.path.isfile('./'+sfile_name) ==True):
        print("exists")
        results.to_csv(sfile_name, mode="a", index=False, header=False)
    else:
        print("No.. so creating")
        results.to_csv(sfile_name,  index=False, header=True) 
    return

from pyriemann.tangentspace import TangentSpace,FGDA
def fgda(X_train,X_test,y_train):
    fb =  FilterBank(make_pipeline(Covariances(estimator="oas"),FGDA()), flatten=False)
    fb.fit(X_train,y_train)
    xt=fb.transform(X_train);xe=fb.transform(X_test)
    return xt,xe,y_train

def load_pkl(sub,nband,fpath):
    direc=fpath #+"/xy/sub"+str(sub)
    with open(direc+'xt_cnn7.pkl', 'rb') as file:
        xt=pickle.load(file)
    with open(direc+'xe_cnn7.pkl', 'rb') as file:
        xe=pickle.load(file)
    with open(direc+'yt_cnn7.pkl', 'rb') as file:
        yt=pickle.load(file)
    with open(direc+'ye_cnn7.pkl', 'rb') as file:
        ye=pickle.load(file)
    print(xt.keys())
    return xt[(sub,nband)],xe[(sub,nband)],yt[(sub,nband)],ye[(sub,nband)]

In [32]:
def model(x, w, w2, w3, w_o, p_keep_conv, p_keep_hidden,b1,b2):
    l1a = tf.nn.relu(tf.nn.conv3d(x, w, strides=[1, 1, 1, 1, 1], padding='VALID'))
    l1a = tf.nn.bias_add(l1a, b1)
    l1a = tf.nn.dropout(l1a, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv3d(l1a, w2, strides=[1, 1, 1, 1, 1], padding='VALID'))
    l2a = tf.nn.bias_add(l2a, b2)
    l2a = tf.nn.dropout(l2a, p_keep_conv)
      

    l2 = tf.reshape(l2a, [-1, w3.get_shape().as_list()[0]])
    l3 = tf.nn.relu(tf.matmul(l2, w3))
    l3 = tf.nn.dropout(l3, p_keep_hidden)

    pyx = tf.matmul(l3, w_o)
    return pyx

def nn_cnn3d(trX,trY,teX,teY,itr,k1,k2,f1,f2,eta,dropout):
    nfb=trX.shape[-1]
    nch=trX.shape[1]
    n_out=trY.shape[-1]
    trX = trX.reshape(-1, nch, nch, nfb, 1) 
    teX = teX.reshape(-1, nch, nch, nfb, 1) 

    x = tf.placeholder("float", [None, nch, nch, nfb, 1]) 
    y_ = tf.placeholder("float", [None, n_out])

    kernel1=k1
    kernel2=k2
    feature_map1=f1 #50
    feature_map2=f2 #100

    w = tf.get_variable("w", shape=[kernel1, kernel1, kernel2, 1, feature_map1],
                        initializer=tf.truncated_normal_initializer(stddev=0.01))
    w2 = tf.get_variable("w2", shape=[nch+1-kernel1, nch+1-kernel1, nfb+1-kernel2, feature_map1, feature_map2],
                         initializer=tf.truncated_normal_initializer(stddev=0.01))
    w3 = tf.get_variable("w3", shape=[feature_map2, feature_map2], initializer=tf.truncated_normal_initializer(stddev=0.01))
    w_o = tf.get_variable("w_o", shape=[feature_map2, n_out], initializer=tf.truncated_normal_initializer(stddev=0.01))

    b1 = tf.get_variable("b1", shape=[feature_map1], initializer=tf.constant_initializer(0.0))
    b2 = tf.get_variable("b2", shape=[feature_map2], initializer=tf.constant_initializer(0.0))

    p_keep_conv = tf.placeholder("float")
    p_keep_hidden = tf.placeholder("float")
    y = model(x, w, w2, w3, w_o, p_keep_conv, p_keep_hidden,b1,b2)

    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1)), tf.float32))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
    train_op = tf.train.AdamOptimizer(eta).minimize(cost)
    predict_op = tf.argmax(y, 1)

    acc = [0] * itr
    tracc = [0] * itr
    # Launch the graph in a session
    ypred={}
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        for i in range(itr):
            sess.run(train_op, feed_dict={x: np.array(trX), y_: np.array(trY), p_keep_conv: dropout, p_keep_hidden: dropout})
            #print(100 * np.mean(np.argmax(teY, axis=1) == sess.run(predict_op, feed_dict={x: teX, y_: teY, p_keep_conv: 1.0, p_keep_hidden: 1.0})))
    #         st = time.time()

            ypred[i]=sess.run(predict_op, feed_dict={x: teX, y_: teY, p_keep_conv: 1.0, p_keep_hidden: 1.0})
            acc[i] = np.mean(np.argmax(teY, axis=1) == ypred[i])
            #en = time.time()
            tracc[i] = sess.run(accuracy, feed_dict={x: trX, y_: trY, p_keep_conv: 0.8, p_keep_hidden: 0.8})
            print(i,tracc[i],acc[i])
    y_pred=ypred[np.argmax(acc)]        
    return y_pred,ypred[i]#,y_pred #,ypred[np.argmax(acc)]



In [44]:
nbs=["4"] #["2468","4","24","246"]
subj = [ 1,2,3,4,5,6,7,8,9]

nbands_Sel=[7] #[2,3,4,5,6,8,9,10]
mode="baseline_fb_fixed7_fgda"
dataset = BNCI2014001();dataset_name="BNCI2014001"
itr=200;k1=5;k2=5;f1=50;f2=100;eta=0.001;dropout=0.6 #itr=200;k1=5;k2=5;f1=50;f2=100;eta=0.001;dropout=0.7

pipe="cnn3d_itr"+str(itr)+"_k1"+str(k1)+"_k2"+str(k2)+"_f1"+str(f1)+"_f2"+str(f2)+"_eta"+str(eta)+"_dropout"+str(dropout)
for nb in nbs:
    fpath="tw_fb"+nb+"/"#fpath="timewin_filters_all/fb"+nb
    fpath="baseline_fb/"
    yp={};y_act_p={}
    for sub in subj:
        for nband in nbands_Sel:
            print("bands_hz:",nb,"|| subject:",sub, "|| nband:",nband)
            xt,xe,yt,ye=load_pkl(sub,nband,fpath)
            print(xt.shape,xe.shape,yt.shape,np.unique(yt))
            xt,xe,yt=fgda(xt,xe,yt)
            trX = xt;      teX = xe
            trY,teY=onehot_encode_list(yt,ye)
            tf.reset_default_graph()
            tf.set_random_seed(1230)
            yp[sub],y_act_p[sub]=nn_cnn3d(trX,trY,teX,teY,itr,k1,k2,f1,f2,eta,dropout)
         

bands_hz: 4 || subject: 1 || nband: 7
dict_keys([(1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7), (8, 7), (9, 7)])
(288, 22, 22, 7) (288, 22, 22, 7) (288,) [0 1 2 3]
0 0.2673611 0.25
1 0.24652778 0.3263888888888889
2 0.28819445 0.25
3 0.27430555 0.25
4 0.2361111 0.25
5 0.28125 0.25
6 0.28125 0.25
7 0.2638889 0.25
8 0.30555555 0.3611111111111111
9 0.2361111 0.25
10 0.2673611 0.25
11 0.2361111 0.25
12 0.2951389 0.25
13 0.2673611 0.25
14 0.25 0.25
15 0.30208334 0.25
16 0.24305555 0.25
17 0.30208334 0.25
18 0.33680555 0.2743055555555556
19 0.30555555 0.3506944444444444
20 0.375 0.3506944444444444
21 0.32291666 0.2743055555555556
22 0.2777778 0.2708333333333333
23 0.32291666 0.3576388888888889
24 0.28125 0.34375
25 0.28819445 0.4513888888888889
26 0.35416666 0.3819444444444444
27 0.3298611 0.3298611111111111
28 0.38194445 0.4895833333333333
29 0.4375 0.4722222222222222
30 0.47916666 0.4618055555555556
31 0.4236111 0.4652777777777778
32 0.44444445 0.5069444444444444
33 0.4027778 0.368

104 0.21527778 0.25
105 0.2638889 0.25
106 0.24305555 0.25
107 0.25 0.25
108 0.26041666 0.25
109 0.27083334 0.25
110 0.23958333 0.25
111 0.22222222 0.25
112 0.24652778 0.25
113 0.2013889 0.25
114 0.24652778 0.25
115 0.25694445 0.25
116 0.24652778 0.25
117 0.26041666 0.25
118 0.26041666 0.25
119 0.2638889 0.25
120 0.25 0.25
121 0.2847222 0.25
122 0.27430555 0.25
123 0.24652778 0.25
124 0.2673611 0.2638888888888889
125 0.23958333 0.25
126 0.2777778 0.25
127 0.24305555 0.25
128 0.24652778 0.25
129 0.2986111 0.25
130 0.24652778 0.25
131 0.22569445 0.25
132 0.27083334 0.25
133 0.2673611 0.25
134 0.24652778 0.25
135 0.2534722 0.25
136 0.2361111 0.25
137 0.22916667 0.25
138 0.23958333 0.25
139 0.22569445 0.25
140 0.2777778 0.25
141 0.29166666 0.25
142 0.28125 0.25
143 0.27430555 0.25
144 0.24652778 0.25
145 0.2673611 0.25
146 0.24305555 0.25
147 0.27430555 0.25
148 0.21875 0.25
149 0.25 0.25
150 0.2534722 0.25
151 0.30208334 0.25
152 0.22916667 0.25
153 0.24652778 0.25
154 0.2326389 0.25
155 

11 0.25 0.25
12 0.2777778 0.25
13 0.25 0.25
14 0.2673611 0.25
15 0.2361111 0.25
16 0.22569445 0.25
17 0.26041666 0.2604166666666667
18 0.23958333 0.25
19 0.28819445 0.25
20 0.2638889 0.25
21 0.22569445 0.25
22 0.2777778 0.25
23 0.2638889 0.25
24 0.25694445 0.25
25 0.22569445 0.25
26 0.28819445 0.25
27 0.2534722 0.25
28 0.25694445 0.25
29 0.2534722 0.25
30 0.23958333 0.25
31 0.25 0.25
32 0.26041666 0.25
33 0.25 0.25
34 0.25694445 0.25
35 0.2673611 0.25
36 0.22222222 0.25
37 0.24305555 0.25
38 0.2534722 0.25
39 0.26041666 0.25
40 0.26041666 0.25
41 0.2361111 0.25
42 0.27083334 0.25
43 0.2777778 0.25
44 0.27430555 0.25
45 0.25 0.25
46 0.23958333 0.25
47 0.2638889 0.25
48 0.27083334 0.25
49 0.24652778 0.25
50 0.27083334 0.25
51 0.27083334 0.25
52 0.25694445 0.25
53 0.25694445 0.25
54 0.27083334 0.25
55 0.2777778 0.25
56 0.29166666 0.25
57 0.27083334 0.25
58 0.26041666 0.2638888888888889
59 0.2534722 0.25
60 0.25694445 0.25
61 0.27430555 0.2569444444444444
62 0.2847222 0.4131944444444444
63

142 0.24652778 0.25
143 0.24305555 0.25
144 0.25694445 0.25
145 0.23958333 0.25
146 0.21875 0.25
147 0.2534722 0.25
148 0.2326389 0.25
149 0.27430555 0.25
150 0.27430555 0.25
151 0.28125 0.25
152 0.22916667 0.25
153 0.27083334 0.25
154 0.27430555 0.25
155 0.24305555 0.25
156 0.26041666 0.25
157 0.2673611 0.25
158 0.19097222 0.25
159 0.24652778 0.25
160 0.2847222 0.25
161 0.2534722 0.25
162 0.25694445 0.25
163 0.2673611 0.25
164 0.24652778 0.25
165 0.27430555 0.25
166 0.3125 0.25
167 0.2847222 0.25
168 0.2638889 0.25
169 0.22569445 0.25
170 0.25694445 0.25
171 0.21180555 0.25
172 0.2013889 0.25
173 0.24305555 0.25
174 0.24305555 0.25
175 0.2534722 0.25
176 0.25694445 0.25
177 0.35416666 0.25
178 0.21180555 0.25
179 0.26041666 0.25
180 0.25 0.25
181 0.25694445 0.25
182 0.2777778 0.25
183 0.28819445 0.25
184 0.2673611 0.25
185 0.24305555 0.25
186 0.26041666 0.25
187 0.28125 0.25
188 0.24305555 0.25
189 0.28125 0.25
190 0.21180555 0.25
191 0.25 0.25
192 0.25 0.25
193 0.2534722 0.25
194 0.1

41 0.25 0.25
42 0.24652778 0.25
43 0.2986111 0.25
44 0.28125 0.25
45 0.2847222 0.25
46 0.3159722 0.2638888888888889
47 0.26041666 0.25
48 0.2777778 0.25
49 0.3159722 0.3819444444444444
50 0.2951389 0.25
51 0.3472222 0.3784722222222222
52 0.33333334 0.25
53 0.3472222 0.4236111111111111
54 0.3888889 0.3611111111111111
55 0.33680555 0.3611111111111111
56 0.34375 0.2743055555555556
57 0.31944445 0.2604166666666667
58 0.26041666 0.25
59 0.33333334 0.4305555555555556
60 0.2847222 0.2534722222222222
61 0.39583334 0.3784722222222222
62 0.26041666 0.25
63 0.29166666 0.25
64 0.3888889 0.3506944444444444
65 0.33680555 0.2881944444444444
66 0.375 0.3229166666666667
67 0.3263889 0.25
68 0.4652778 0.3958333333333333
69 0.39930555 0.3402777777777778
70 0.45833334 0.3993055555555556
71 0.38541666 0.2569444444444444
72 0.4236111 0.4340277777777778
73 0.4548611 0.4027777777777778
74 0.38541666 0.3263888888888889
75 0.4409722 0.3333333333333333
76 0.4201389 0.4340277777777778
77 0.4722222 0.3611111111111

107 0.8923611 0.6701388888888888
108 0.8402778 0.6666666666666666
109 0.8611111 0.6701388888888888
110 0.8611111 0.65625
111 0.8645833 0.6458333333333334
112 0.8611111 0.6701388888888888
113 0.8541667 0.6631944444444444
114 0.8645833 0.6666666666666666
115 0.8472222 0.6458333333333334
116 0.8784722 0.6736111111111112
117 0.8680556 0.6631944444444444
118 0.8680556 0.6631944444444444
119 0.8541667 0.65625
120 0.8680556 0.6631944444444444
121 0.8784722 0.6597222222222222
122 0.8645833 0.6666666666666666
123 0.8645833 0.6736111111111112
124 0.8506944 0.6736111111111112
125 0.8611111 0.6875
126 0.8611111 0.6666666666666666
127 0.8611111 0.6736111111111112
128 0.8576389 0.6666666666666666
129 0.8472222 0.6597222222222222
130 0.8472222 0.6527777777777778
131 0.8576389 0.6597222222222222
132 0.8506944 0.6666666666666666
133 0.8854167 0.6875
134 0.8541667 0.6875
135 0.8576389 0.6736111111111112
136 0.9027778 0.6805555555555556
137 0.8402778 0.6631944444444444
138 0.8680556 0.6770833333333334
13

172 0.7708333 0.65625
173 0.78125 0.6840277777777778
174 0.7604167 0.7083333333333334
175 0.7395833 0.6979166666666666
176 0.7395833 0.6770833333333334
177 0.7534722 0.6805555555555556
178 0.7708333 0.6979166666666666
179 0.7777778 0.7013888888888888
180 0.7638889 0.7048611111111112
181 0.7604167 0.7118055555555556
182 0.78125 0.6875
183 0.7708333 0.6805555555555556
184 0.78125 0.6631944444444444
185 0.7604167 0.7083333333333334
186 0.7638889 0.7048611111111112
187 0.7881944 0.7013888888888888
188 0.7465278 0.7048611111111112
189 0.7777778 0.6979166666666666
190 0.7777778 0.6944444444444444
191 0.7986111 0.6944444444444444
192 0.78125 0.6840277777777778
193 0.78125 0.7048611111111112
194 0.78125 0.6944444444444444
195 0.7743056 0.6979166666666666
196 0.7743056 0.6770833333333334
197 0.7777778 0.6805555555555556
198 0.7673611 0.6875
199 0.7881944 0.7083333333333334


In [40]:
df = pd.DataFrame(columns=['dataset','session','channels','subject','n_filters','pipeline','acc',
    'recall','precision','f1','kappa','sensitivity','specificity'])
sfile_name = fpath+"CNN_accResults_CrossSess_"+mode+"_dscore_nbands"+nb+".csv"    
nchan=xt.shape[1]
for sub in subj:
    acc,recall,precision,f1,kappa,sens,spec=all_metrics(ye,yp[sub])
    df.loc[len(df)] = [pipe, "session_E", nchan, sub, nband, "3DCNN", acc,recall,precision,f1,kappa,sens,spec]
save_csvfile(df,sfile_name)

exists


In [41]:
df


,dataset,session,channels,subject,n_filters,pipeline,acc,recall,precision,f1,kappa,sensitivity,specificity
0,cnn3d_itr200_k15_k25_f150_f2100_eta0.001_dropo...,session_E,22,4,7,3DCNN,0.486111,0.486111,0.486111,0.486111,0.314815,0.486111,0.828704


In [42]:
df = pd.DataFrame(columns=['dataset','session','channels','subject','n_filters','pipeline','acc',
    'recall','precision','f1','kappa','sensitivity','specificity'])
sfile_name = fpath+"CNN_act_accResults_CrossSess_"+mode+"_dscore_nbands"+nb+".csv"    
nchan=xt.shape[1]
for sub in subj:
    acc,recall,precision,f1,kappa,sens,spec=all_metrics(ye,y_act_p[sub])
    df.loc[len(df)] = [pipe, "session_E", nchan, sub, nband, "3DCNN", acc,recall,precision,f1,kappa,sens,spec]
save_csvfile(df,sfile_name)

exists


In [43]:
df

,dataset,session,channels,subject,n_filters,pipeline,acc,recall,precision,f1,kappa,sensitivity,specificity
0,cnn3d_itr200_k15_k25_f150_f2100_eta0.001_dropo...,session_E,22,4,7,3DCNN,0.4375,0.4375,0.4375,0.4375,0.25,0.4375,0.8125
